In [1]:
import sys
sys.path.append('../')
from pathlib import Path
import pandas as pd
import numpy as np

from scripts.base import Feature, get_arguments, generate_features
Feature.dir = "F:/Kaggle/ventilator-pressure-prediction/data/input/features"

In [19]:
INPUT_DIR = Path("F:/Kaggle/ventilator-pressure-prediction/data/input/")
OUTPUT_DIR = Path('F:/Kaggle/ventilator-pressure-prediction/data/output/')
FEAT_DIR = Path("F:/Kaggle/ventilator-pressure-prediction/data/input/features/")

def get_train_test():
    _train = pd.read_csv(INPUT_DIR / "train.csv")
    _test = pd.read_csv(INPUT_DIR / "test.csv")
    return _train, _test

def load_datasets(feats):
    dfs = [pd.read_feather(FEAT_DIR / f'{f}_train.ftr') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    dfs = [pd.read_feather(FEAT_DIR / f'{f}_test.ftr') for f in feats]
    X_test = pd.concat(dfs, axis=1)
    return X_train, X_test

In [41]:
class Base(Feature):
    """結合用のデータ"""
    def create_features(self):
        self.train = train[['id', 'breath_id', 'time_step', 'u_in', 'u_out', 'pressure', 'fold']].copy()
        self.test = test[['id', 'breath_id', 'time_step', 'u_in', 'u_out']].copy()
        
train, test = get_train_test()
train_v2 = pd.read_csv(INPUT_DIR / "train_v2.csv")
train = pd.concat([train, train_v2[["fold"]]],axis=1)
Base().run().save()

[Base] start
[Base] done in 0 s


In [32]:
class Area(Feature):
    """u_inの下の面積っぽいけどなんか式違いそう"""
    def create_features(self):
        train['area'] = train['time_step'] * train['u_in'] 
        self.train['area'] = train.groupby('breath_id')['area'].cumsum()
        test['area'] = test['time_step'] * test['u_in'] 
        self.test['area'] = test.groupby('breath_id')['area'].cumsum()
train, test = get_train_test()
Area().run().save()

[Area] start
[Area] done in 1 s


In [34]:
class Cross(Feature):
    """よくわからない"""
    def create_features(self):
        self.train['cross']  = train['u_in']*train['u_out'] 
        self.train['cross2'] = train['time_step']*train['u_out']
        self.test['cross']  = test['u_in']*test['u_out']
        self.test['cross2'] = test['time_step']*test['u_out']
        
train, test = get_train_test()
Cross().run().save()

[Cross] start
[Cross] done in 1 s


In [38]:
class U_in_cumsum_mean(Feature):
    """累積和と累積平均"""
    def create_features(self):
        def create(df):
            df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()# 累積和
            df['one'] = 1
            df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
            df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
            return df
        self.train = create(train)[['u_in_cumsum', 'u_in_cummean']]
        self.test = create(test)[['u_in_cumsum', 'u_in_cummean']]
        
train, test = get_train_test()
U_in_cumsum_mean().run().save()

[U_in_cumsum_mean] start
[U_in_cumsum_mean] done in 1 s


In [15]:
class U_in_Lag(Feature):
    """累積和と累積平均"""
    def create_features(self):
        def create(df):
            df['u_in_lag'] = df.groupby('breath_id')['u_in'].shift(1)
            df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
            df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
            df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
            df['u_in_lag_back'] = df.groupby('breath_id')['u_in'].shift(-1)
            df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
            df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
            df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
            df = df.fillna(0)
            return df
        cols = ['u_in_lag','u_in_lag2','u_in_lag3','u_in_lag4','u_in_lag_back','u_in_lag_back2','u_in_lag_back3','u_in_lag_back4']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
U_in_Lag().run().save()

[U_in_Lag] start
[U_in_Lag] done in 3 s


In [16]:
class U_out_Lag(Feature):
    """累積和と累積平均"""
    def create_features(self):
        def create(df):
            df['u_out_lag'] = df.groupby('breath_id')['u_out'].shift(1)
            df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
            df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
            df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
            df['u_out_lag_back'] = df.groupby('breath_id')['u_out'].shift(-1)
            df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
            df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
            df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
            df = df.fillna(0)
            return df
        cols = ['u_out_lag','u_out_lag2','u_out_lag3','u_out_lag4','u_out_lag_back','u_out_lag_back2','u_out_lag_back3','u_out_lag_back4']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
U_out_Lag().run().save()

[U_out_Lag] start
[U_out_Lag] done in 3 s


In [67]:
class RC_OHE(Feature):
    """
    RとCのOHE
    """
    def create_features(self):
        def create(df):
            df['R'] = df['R'].astype(str)
            df['C'] = df['C'].astype(str)
            df['RC'] = df['R']+df['C']
            df = pd.get_dummies(df)
            return df
        cols = ['R_20', 'R_5', 'R_50', 'C_10', 'C_20', 'C_50', 'RC_2010', 'RC_2020', 'RC_2050', 'RC_5010', 'RC_5020', 'RC_5050', 'RC_510', 'RC_520', 'RC_550']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
RC_OHE().run().save()

[RC_OHE] start
[RC_OHE] done in 16 s


In [10]:
class U_out_stat(Feature):
    """
    u_outが0のときと1のときのu_inのmean,std,max
    breath_idごとに値はおなじになる
    """
    def create_features(self):
        def create(df):
            df = df.merge(
                df[df["u_out"]==0].groupby('breath_id')['u_in'].agg(["mean", "std", "max"]).add_prefix("u_out0_").reset_index(),
                on="breath_id"
                )
            df = df.merge(
                df[df["u_out"]==1].groupby('breath_id')['u_in'].agg(["mean", "std", "max"]).add_prefix("u_out1_").reset_index(),
                on="breath_id"
                )
            return df
        cols = ['u_out0_mean', 'u_out0_max', 'u_out0_std', 'u_out1_mean', 'u_out1_max', 'u_out1_std']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
U_out_stat().run().save()

[U_out_stat] start
[U_out_stat] done in 7 s


In [47]:
class U_inout_max(Feature):
    """
    u_outが0のときと1のときのu_inのmean,std,max
    breath_idごとに値はおなじになる
    """
    def create_features(self):
        def create(df):
            df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
            df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
            return df
        cols = ['breath_id__u_in__max', 'breath_id__u_out__max']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
U_inout_max().run().save()

[U_inout_max] start
[U_inout_max] done in 1 s


In [13]:
class Time(Feature):
    """
    """
    def create_features(self):
        def create(df):
            # breath_time
            df['breath_time'] = df['time_step'] - df['time_step'].shift(1)
            df.loc[df['time_step'] == 0, 'breath_time'] = 0
            # u_in_time
            df['u_in_time'] = df['u_in'] - df['u_in'].shift(1)
            df.loc[df['time_step'] == 0, 'u_in_time'] = 0
            return df
        cols = ['breath_time', 'u_in_time']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
Time().run().save()

[Time] start
[Time] done in 1 s


In [21]:
class U_in_Lag_Diff(Feature):
    """
    """
    def create_features(self):
        def create(df):
            df["u_in_lag1_diff"] = df["u_in_lag"] - df["u_in"]
            df["u_in_lag2_diff"] = df["u_in_lag2"] - df["u_in"]
            df["u_in_lag3_diff"] = df["u_in_lag3"] - df["u_in"]
            df["u_in_lag4_diff"] = df["u_in_lag4"] - df["u_in"]
            return df
        cols = ['u_in_lag1_diff', 'u_in_lag2_diff','u_in_lag3_diff', 'u_in_lag4_diff']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        display(self.train)

feats = ['Base', 'U_in_Lag']
train, test = load_datasets(feats)
display(train)
U_in_Lag_Diff().run().save()

,id,breath_id,time_step,u_in,u_out,pressure,fold,u_in_lag,u_in_lag2,u_in_lag3,u_in_lag4,u_in_lag_back,u_in_lag_back2,u_in_lag_back3,u_in_lag_back4
0,1,1,0.000000,0.080043,0,5.837492,4,0.000000,0.000000,0.000000,0.000000,18.383041,22.509278,22.808822,25.355850
1,2,1,0.033652,2.964399,0,5.907794,4,0.083334,0.000000,0.000000,0.000000,22.509278,22.808822,25.355850,27.259866
2,3,1,0.067514,3.157395,0,7.876254,4,18.383041,0.083334,0.000000,0.000000,22.808822,25.355850,27.259866,27.127486
3,4,1,0.101542,3.170056,0,11.742872,4,22.509278,18.383041,0.083334,0.000000,25.355850,27.259866,27.127486,26.807732
4,5,1,0.135756,3.271690,0,12.234987,4,22.808822,22.509278,18.383041,0.083334,27.259866,27.127486,26.807732,27.864715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,6035996,125749,2.504603,0.912168,1,3.869032,0,1.420711,1.353205,1.357586,1.362261,1.488497,1.558978,1.272663,1.482739
6035996,6035997,125749,2.537961,0.911679,1,3.869032,0,1.489714,1.420711,1.353205,1.357586,1.558978,1.272663,1.482739,0.000000
6035997,6035998,125749,2.571408,0.939608,1,3.798729,0,1.488497,1.489714,1.420711,1.353205,1.272663,1.482739,0.000000,0.000000
6035998,6035999,125749,2.604744,0.820952,1,4.079938,0,1.558978,1.488497,1.489714,1.420711,1.482739,0.000000,0.000000,0.000000


[U_in_Lag_Diff] start


,u_in_lag1_diff,u_in_lag2_diff,u_in_lag3_diff,u_in_lag4_diff
0,-0.080043,-0.080043,-0.080043,-0.080043
1,-2.881065,-2.964399,-2.964399,-2.964399
2,15.225646,-3.074061,-3.157395,-3.157395
3,19.339222,15.212985,-3.086722,-3.170056
4,19.537132,19.237587,15.111351,-3.188356
...,...,...,...,...
6035995,0.508544,0.441037,0.445419,0.450094
6035996,0.578035,0.509033,0.441526,0.445908
6035997,0.548889,0.550106,0.481104,0.413597
6035998,0.738025,0.667544,0.668761,0.599759


[U_in_Lag_Diff] done in 1 s


In [44]:
class U_in_Rolling(Feature):
    """
    """
    def create_features(self):
        def create(df):
            df["u_in_rolling_mean2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).mean()["u_in"].reset_index(drop=True)
            df["u_in_rolling_mean4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).mean()["u_in"].reset_index(drop=True)
            df["u_in_rolling_mean10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).mean()["u_in"].reset_index(drop=True)
            
            df["u_in_rolling_max2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).max()["u_in"].reset_index(drop=True)
            df["u_in_rolling_max4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).max()["u_in"].reset_index(drop=True)
            df["u_in_rolling_max10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).max()["u_in"].reset_index(drop=True)
            df["u_in_rolling_min2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).min()["u_in"].reset_index(drop=True)
            df["u_in_rolling_min4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).min()["u_in"].reset_index(drop=True)
            df["u_in_rolling_min10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).min()["u_in"].reset_index(drop=True)
            df["u_in_rolling_std2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).std()["u_in"].reset_index(drop=True)
            df["u_in_rolling_std4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).std()["u_in"].reset_index(drop=True)
            df["u_in_rolling_std10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).std()["u_in"].reset_index(drop=True)
            for col in df.columns:
                df[col] = df[col].fillna(df[col].mean())
            return df
        cols = ['u_in_rolling_mean2', 'u_in_rolling_mean4','u_in_rolling_mean10', 'u_in_rolling_max2', 'u_in_rolling_max4', 'u_in_rolling_max10',
                'u_in_rolling_min2', 'u_in_rolling_min4', 'u_in_rolling_min10', 'u_in_rolling_std2', 'u_in_rolling_std4', 'u_in_rolling_std10']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        display(self.train)

train, test = get_train_test()
U_in_Rolling().run().save()

[U_in_Rolling] start


,u_in_rolling_mean2,u_in_rolling_mean4,u_in_rolling_mean10,u_in_rolling_max2,u_in_rolling_max4,u_in_rolling_max10,u_in_rolling_min2,u_in_rolling_min4,u_in_rolling_min10,u_in_rolling_std2,u_in_rolling_std4,u_in_rolling_std10
0,7.296453,7.122734,6.673223,8.130658,9.024859,10.262890,6.462249,5.530754,4.011267,1.179743,1.644297,2.304005
1,9.233188,7.122734,6.673223,18.383041,9.024859,10.262890,0.083334,5.530754,4.011267,12.939847,1.644297,2.304005
2,20.446160,7.122734,6.673223,22.509278,9.024859,10.262890,18.383041,5.530754,4.011267,2.917690,1.644297,2.304005
3,22.659050,15.946119,6.673223,22.808822,22.808822,10.262890,22.509278,0.083334,4.011267,0.211810,10.766279,2.304005
4,24.082336,22.264248,6.673223,25.355850,25.355850,10.262890,22.808822,18.383041,4.011267,1.801021,2.885502,2.304005
...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,1.455213,1.405304,1.346222,1.489714,1.489714,1.489714,1.420711,1.353205,1.184357,0.048792,0.064171,0.085995
6035996,1.489105,1.438032,1.376636,1.489714,1.489714,1.489714,1.488497,1.353205,1.237674,0.000860,0.065098,0.075534
6035997,1.523737,1.489475,1.400848,1.558978,1.558978,1.558978,1.488497,1.420711,1.237674,0.049838,0.056451,0.091385
6035998,1.415821,1.452463,1.404347,1.558978,1.558978,1.558978,1.272663,1.272663,1.272663,0.202455,0.124311,0.084882


[U_in_Rolling] done in 165 s


In [46]:
feats = ['Base', 'U_in_Rolling']
train, test = load_datasets(feats)
train.head(50)
#pd.read_feather(INPUT_DIR / "features" / "Base_train.ftr").head(20)

,id,breath_id,time_step,u_in,u_out,pressure,fold,u_in_rolling_mean2,u_in_rolling_mean4,u_in_rolling_mean10,u_in_rolling_max2,u_in_rolling_max4,u_in_rolling_max10,u_in_rolling_min2,u_in_rolling_min4,u_in_rolling_min10,u_in_rolling_std2,u_in_rolling_std4,u_in_rolling_std10
0,1,1,0.000000,0.083334,0,5.837492,4,7.296453,7.122734,6.673223,8.130658,9.024859,10.262890,6.462249,5.530754,4.011267,1.179743e+00,1.644297e+00,2.304005
1,2,1,0.033652,18.383041,0,5.907794,4,9.233188,7.122734,6.673223,18.383041,9.024859,10.262890,0.083334,5.530754,4.011267,1.293985e+01,1.644297e+00,2.304005
2,3,1,0.067514,22.509278,0,7.876254,4,20.446160,7.122734,6.673223,22.509278,9.024859,10.262890,18.383041,5.530754,4.011267,2.917690e+00,1.644297e+00,2.304005
3,4,1,0.101542,22.808822,0,11.742872,4,22.659050,15.946119,6.673223,22.808822,22.808822,10.262890,22.509278,0.083334,4.011267,2.118099e-01,1.076628e+01,2.304005
4,5,1,0.135756,25.355850,0,12.234987,4,24.082336,22.264248,6.673223,25.355850,25.355850,10.262890,22.808822,18.383041,4.011267,1.801021e+00,2.885502e+00,2.304005
5,6,1,0.169698,27.259866,0,12.867706,4,26.307858,24.483454,6.673223,27.259866,27.259866,10.262890,25.355850,22.509278,4.011267,1.346342e+00,2.248800e+00,2.304005
6,7,1,0.203708,27.127486,0,14.695562,4,27.193676,25.638006,6.673223,27.259866,27.259866,10.262890,27.127486,22.808822,4.011267,9.360695e-02,2.076284e+00,2.304005
7,8,1,0.237723,26.807732,0,15.890699,4,26.967609,26.637733,6.673223,27.127486,27.259866,10.262890,26.807732,25.355850,4.011267,2.261001e-01,8.754103e-01,2.304005
8,9,1,0.271776,27.864715,0,15.539188,4,27.336224,27.264950,6.673223,27.864715,27.864715,10.262890,26.807732,26.807732,4.011267,7.474003e-01,4.426019e-01,2.304005
9,10,1,0.305732,28.313036,0,15.750094,4,28.088876,27.528242,22.651316,28.313036,28.313036,28.313036,27.864715,26.807732,0.083334,3.170104e-01,6.852874e-01,8.515846


In [49]:
train, test = get_train_test()
df = train.copy()

df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
df = df.fillna(0)

In [50]:
df[70:90]

,id,breath_id,R,C,time_step,u_in,u_out,pressure,u_out_lag3,u_out_lag_back3
70,71,1,20,50,2.383458,4.940237,1,6.751420,1.0,1.0
71,72,1,20,50,2.417553,4.949604,1,6.540513,1.0,1.0
72,73,1,20,50,2.451473,4.957466,1,6.399909,1.0,1.0
73,74,1,20,50,2.485587,4.964136,1,6.751420,1.0,1.0
74,75,1,20,50,2.519502,4.969730,1,6.470211,1.0,1.0
75,76,1,20,50,2.553593,4.974474,1,6.399909,1.0,1.0
76,77,1,20,50,2.587754,4.978481,1,6.610815,1.0,1.0
77,78,1,20,50,2.621773,4.981847,1,6.329607,1.0,NaN
78,79,1,20,50,2.655746,4.984683,1,6.540513,1.0,NaN
79,80,1,20,50,2.689766,4.987079,1,6.470211,1.0,NaN


In [53]:
train, test = get_train_test()
df = train.copy()

,id,breath_id,time_step,u_in,u_out,pressure,R_5,R_20,R_50,C_10,...,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,1,0.000000,0.083334,0,5.837492,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
1,2,1,0.033652,18.383041,0,5.907794,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
2,3,1,0.067514,22.509278,0,7.876254,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
3,4,1,0.101542,22.808822,0,11.742872,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
4,5,1,0.135756,25.355850,0,12.234987,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,6035996,125749,2.504603,1.489714,1,3.869032,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
6035996,6035997,125749,2.537961,1.488497,1,3.869032,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
6035997,6035998,125749,2.571408,1.558978,1,3.798729,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
6035998,6035999,125749,2.604744,1.272663,1,4.079938,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0


In [59]:
df['hoge_lag1'] = df.groupby('hoge_id')['hoge'].shift(1)

In [35]:
train = pd.read_csv(INPUT_DIR / "train_v2.csv")
train

,id,breath_id,R,C,time_step,u_in,u_out,pressure,breath_time,u_in_time,fold,u_in_cumsum,u_in_lag2
0,1,1,1,2,0.000000,0.080043,0,5.837492,0.000000,0.000000,4,0.083334,0.000000
1,2,1,1,2,0.033652,2.964399,0,5.907794,0.033652,2.884355,4,18.466375,0.000000
2,3,1,1,2,0.067514,3.157395,0,7.876254,0.033862,0.192997,4,40.975653,0.083334
3,4,1,1,2,0.101542,3.170056,0,11.742872,0.034028,0.012661,4,63.784476,18.383041
4,5,1,1,2,0.135756,3.271690,0,12.234987,0.034213,0.101634,4,89.140326,22.509278
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,6035996,125749,2,0,2.504603,0.912168,1,3.869032,0.033412,0.028106,0,387.109297,1.353205
6035996,6035997,125749,2,0,2.537961,0.911679,1,3.869032,0.033358,-0.000489,0,388.597794,1.420711
6035997,6035998,125749,2,0,2.571408,0.939608,1,3.798729,0.033447,0.027929,0,390.156772,1.489714
6035998,6035999,125749,2,0,2.604744,0.820952,1,4.079938,0.033337,-0.118656,0,391.429435,1.488497
